In [1]:
import tensorflow as tf
import os
import cv2
import imghdr

In [2]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'
!chmod 600 /content/kaggle.json

In [ ]:
!kaggle datasets download -d carlosrunner/pizza-not-pizza -p /content/

In [4]:
import zipfile

In [5]:
with zipfile.ZipFile('/content/pizza-not-pizza.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/pizza-not-pizza')

In [6]:
data_dir = '/content/pizza-not-pizza/pizza_not_pizza'

In [ ]:
files = os.listdir(data_dir)
print(files)

In [8]:
os.remove("/content/pizza-not-pizza/pizza_not_pizza/food101_subset.py")

In [9]:
image_exts = ['jpeg','jpg', 'bmp', 'png']

In [10]:
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))

In [11]:
import numpy as np
from matplotlib import pyplot as plt

In [12]:
data = tf.keras.utils.image_dataset_from_directory(data_dir)

Found 1966 files belonging to 2 classes.


In [13]:
class_names = data.class_names
print(class_names)

['not_pizza', 'pizza']


In [14]:
data_iterator = data.as_numpy_iterator()
batch = data_iterator.next()

In [15]:
batch[0].shape

(32, 256, 256, 3)

In [16]:
batch[1]

array([1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 1, 1], dtype=int32)

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

# Pizza ----> 1  
# Not Pizza ----> 0

In [18]:
data = data.map(lambda x,y: (x/255, y))

In [ ]:
data.as_numpy_iterator().next()

In [20]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1) + 1

In [21]:
len(data)

62

In [22]:
train_size + val_size + test_size

62

In [23]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [25]:
model = Sequential()

In [26]:
model.add(Conv2D(128, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [27]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

In [29]:
logdir='/content/logs'

In [30]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 5))

# Plot Loss
axs[0].plot(hist.history['loss'], color='teal', label='loss')
axs[0].plot(hist.history['val_loss'], color='orange', label='val_loss')
axs[0].set_title('Loss', fontsize=16)
axs[0].legend(loc="upper right")

# Plot Accuracy
axs[1].plot(hist.history['accuracy'], color='teal', label='accuracy')
axs[1].plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
axs[1].set_title('Accuracy', fontsize=16)
axs[1].legend(loc="lower right")

plt.show()

In [33]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [34]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [36]:
print("Precision: {:.4f}, Recall: {:.4f}, Accuracy: {:.4f}".format(pre.result(), re.result(), acc.result()))

Precision: 0.7632, Recall: 0.8614, Accuracy: 0.8010


In [ ]:
img_bgr1 = cv2.imread('/content/testpizza.jpeg')
img_rgb1 = cv2.cvtColor(img_bgr1, cv2.COLOR_BGR2RGB)
plt.imshow(img_rgb1)
plt.show()

In [ ]:
resize1 = tf.image.resize(img_rgb1, (256,256))
plt.imshow(resize1.numpy().astype(int))
plt.show()

In [39]:
yhat1 = model.predict(np.expand_dims(resize1/255, 0))

1/1 [==============================] - 0s 148ms/step


In [40]:
yhat1

array([[0.99208003]], dtype=float32)

In [41]:
if yhat1 > 0.5: 
    print(f'Predicted class is Pizza')
else:
    print(f'Predicted class is NOT Pizza')

Predicted class is Pizza


In [ ]:
img_bgr2 = cv2.imread('/content/testnotpizza.jpeg')
img_rgb2 = cv2.cvtColor(img_bgr2, cv2.COLOR_BGR2RGB)
plt.imshow(img_rgb2)
plt.show()

In [ ]:
resize2 = tf.image.resize(img_rgb2, (256,256))
plt.imshow(resize2.numpy().astype(int))
plt.show()

In [44]:
yhat2 = model.predict(np.expand_dims(resize2/255, 0))

1/1 [==============================] - 0s 36ms/step


In [45]:
yhat2

array([[0.21916233]], dtype=float32)

In [46]:
if yhat2 > 0.5: 
    print(f'Predicted class is Pizza')
else:
    print(f'Predicted class is NOT Pizza')

Predicted class is NOT Pizza


In [47]:
from tensorflow.keras.models import load_model

In [48]:
model.save(os.path.join('models','pizzaclassifier.h5'))

In [49]:
new_model = load_model('/content/models/pizzaclassifier.h5')

In [50]:
new_model.predict(np.expand_dims(resize1/255, 0))

1/1 [==============================] - 0s 98ms/step


array([[0.99208003]], dtype=float32)

In [51]:
!zip -r models.zip /content/models

  adding: content/models/ (stored 0%)
  adding: content/models/pizzaclassifier.h5 (deflated 15%)


In [52]:
from google.colab import files
files.download('models.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>